#### excel封装
一、 excel操作

日常人工操作excel：
1. 打开excel表  work_book
2. 选择sheet   sheet
3. 选择单元格  cell

二、第三方库openpyxl
1. 安装openpyxl:pip install -i https://pypi.tuna.tsinghua.edu.cn/simple openpyxl
2. 读取load_workbook()  
    源码：def load_workbook(filename, read_only=False, keep_vba=KEEP_VBA,data_only=False, keep_links=True)  
    1. filename:excel文件名（路径）
    2. read_only=False：是否只读，flase:可以读可以写  true：只可以读
    3. keep_vba=KEEP_VBA: 是否保留VBA源码
    4. data_only=False【不用管】：true：读取公式计算结算后的结果  false：默认值，不会读取
    5. keep_links=True【不用管】：是否保留外部链接

In [4]:
import os
from openpyxl import load_workbook

#openpyxl 行和列索引从1开始
#filename中的路径是同目录下的，如果不是同目录下的使用os方法，且支持xlsx格式文件
res_path=os.path.abspath('testCase.xlsx')
work_book=load_workbook(filename=res_path)
#获取sheet对象
sheet=work_book['Sheet1']
#获取单元格数据
val=sheet.cell(1,2).value
print(val)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\lemon\\Python_Learn\\notes\\testCase.xlsx'

3. 获取最大的行，最大的列  
max_rows=sheet.max_row  
max_cols=sheet.max_column  
4. 写表  


In [ ]:
from openpyxl import load_workbook

work_book=load_work('testCase.xlsx')
sheet=work_book['Sheet1']
#写表
sheet.cell(6,1).value="005"

#保存
work_book.save(filename=res_path)

补充：接口测试流程
1. 编写测试用例
2. 读取测试用例
3. 发请求
4. 获取响应结果
5. 和预期结果进行比对（断言）
6. 保存测试结果

获取单元数据  
sheet.iter_rows()  
min_row=None：起始行，默认为1    
max_row=None：结束行，默认是最大的行  
min_col=None：起始列  
max_col=None：结束列  
values_only：默认是false，返回单元格的对象，true：返回单元格内容· 

In [ ]:
from openpyxl import load_workbook
work_book=load_workbook(filename='testCase.xlsx')
sheet=work_book['sheet1']

#获取单元数据
dates=sheet.iter_rows(values_only=True)
#每一行是一个元组
print(list(dates))

封装excel表格  
1. 初始化表格对象
2. 初始化sheet对象
3. 获取表格数据

1. 获取测试用例
2. 测试用例传入接口测试
3. 保存测试结果

In [ ]:
# -*- coding: utf-8 -*-
# @Time: 2021/5/21 10:46
# @Author: xueli
# @File: Demo1602.py
import pprint

import requests
from openpyxl import load_workbook

class LoginCase:
    def __init__(self):
        self.headers={"X-Lemonban-Media-Type":"lemonban.v1","Content-Type":"application/json"}

    #用户登录
    def user_login(self,url,data):
        '''
        :param url: 接口地址
        :param data: 接口参数
        :return:
        '''
        #url='http://api.lemonban.com/futureloan/member/login'
        #data={'mobile_phone':'18820992515','pwd':'Aa123456'}
        res=requests.post(url=url,json=data,headers=self.headers)
        # print(res.json)
        return res.json()

class HandleExcel:
    def __init__(self, file_path, sheet_name):
        # 初始化表格对象
        self.work_book = load_workbook(filename=file_path)
        # 初始化sheet对象
        self.sheet = self.work_book[sheet_name]

    # 获取表头和所有测试用例数据
    def get_all_data(self):
        all_datas = list(self.sheet.iter_rows(values_only=True))
        # print(list(all_datas))
        title = all_datas[0]  # 获取表头
        case_dates = all_datas[1:]  # 获取测试用例数据，list切片
        return title, case_dates

    # 数据拼接（dict）
    def get_case_data_dict(self):
        case_list = []
        title, case_datas = self.get_all_data()
        for i in case_datas:
            res = dict(zip(title, i))
            # print(res)
            case_list.append(res)
        return case_list
    # zip函数使用
    # def test(self):
    #     list1 = ['key1','key2','key3']
    #     list2 = ['val1','val2','val3']
    #     result = zip(list1,list2)
    #     print(dict(result))

if __name__ == "__main__":
    cl = HandleExcel(file_path='testCase.xlsx', sheet_name='sheet1')
    # cl.get_case_data_dict()
    case_list = cl.get_case_data_dict()
    #print(case_list)
    pprint.pprint(case_list)


In [3]:
#zip函数使用
     
list1 = ['key1','key2','key3']
list2 = ['val1','val2','val3']
result = zip(list1,list2)
print(dict(result))

{'key1': 'val1', 'key2': 'val2', 'key3': 'val3'}


In [ ]:
import ast

from Day16.Demo1602 import HandleExcel
from Day16.Demo1602 import LoginCase

handle_excel= HandleExcel(file_path='testCase.xlsx', sheet_name='sheet1')
login_test=LoginCase()

#获取excel测试用例
excel_data=handle_excel.get_case_data_dict()

#发送post请求
for case in excel_data:
    
    #ast方法将字符串变为json数据
    res=login_test.user_login(url=case['url'],data=ast.literal_eval(case['data']))
    print(res)

需求：从excel中读取出测试用例，格式必须是字典，方便维护  
会什么：
1. 会读取excel
2. for循环
3. 类、函数、类属性、实例属性、私有方法、私有属性、列表、字典相关操作

思考：  
1. 从excel获取数据、表头是key,表体是测试用例数据
    1. 导入from openpyxl import load_workbook
    2. 实例化表对象work_book
    3. 初始化sheet对象
    4. 拿表头（key）、拿表体（value）
    面向对象原则：一个方法做一件事
2. 拼接表头和表体，做成dict格式
    1. 使用zip函数进行数据拼接
3. 将测试数据放到业务逻辑里面去执行
    1. 执行测试用例